This file is a sort of testpad for developing stochastic aspects of the model.

In [98]:
import numpy as np
import random
import xpress as xp
xp.init('C:/Program Files/xpressmp/bin/xpauth_personal.xpr') # For licensing.

import networkx as nx
import time

In [99]:
# # Example input data 1

# pairs = ["P1", "P2", "P3", "P4"] 
# altruistic_donors = ["NDD1"]
# nodes = pairs + altruistic_donors
# edges = {("NDD1", "P1"): 2,
#          ("P1", "P2"): 10, 
#          ("P2", "P3"): 10,
#          ("P3", "P4"): 10,
#          ("P4", "P1"): 10
# }

# # Example input data 2

# pairs = ["P1", "P2", "P3", "P4", "P5"] 
# altruistic_donors = ["NDD1"]
# nodes = pairs + altruistic_donors
# edges = {("NDD1", "P1"): 0.1,
#          ("P1", "P2"): 10, 
#          ("P2", "P3"): 9,
#          ("P3", "P4"): 8,
#          ("P4", "P5"): 7,
#          ("P5", "P1"): 6
# }

# Example input data 3

pairs = ["P1", "P2", "P3", "P4", "P5"] 
altruistic_donors = ["NDD1"]
nodes = pairs + altruistic_donors
edges = {("NDD1", "P1"): 0.1,
         ("P1", "P2"): 10, 
         ("P2", "P3"): 9,
         ("P3", "P4"): 8,
         ("P4", "P5"): 7
}


In [100]:
P_dropout_pairs = 0.1
P_dropout_altruistic = 0.1

In [101]:
deleted_nodes = [pair for pair in pairs if np.random.uniform() <= P_dropout_pairs] +\
                [NDD for NDD in altruistic_donors if np.random.uniform() <= P_dropout_altruistic]

In [102]:
deleted_nodes

[]

In [124]:
def get_S(edges, deleted_nodes):
    ''' Takes edges and deleted_nodes, returns S encoding which edges survive.
    S is a dictionary: the keys are the edges of the compatibility graph.
    S[e] is 1 if the edge e survives the deletion of nodes and 0 otherwise.
    '''
    
    S = {e : 0 if e[0] in deleted_nodes or e[1] in deleted_nodes else 1 for e in edges}
    return S

In [125]:
def generate_scenarios(pairs, altruistic_donors, edges):
    """
    Generate a list of dictionaries which encode edge survival.

    Args:
        pairs: list
        altruistic_donors: list
        edges: dictionary (keys: 2-tuple (in/out vx); values: weight of edge)
        N: number of scenarios to generate

    Returns:
        scenarios: a list of dictionaries. Each dictionary is a scenario; in
            each dictionary, keys are edges, value is 1 if edge survives else 0.
    """

    nodes = pairs+altruistic_donors

    scenarios = []

    # DELETE one node:
    # PROOF OF CONCEPT!
    # TODO: extend so that the deletion of nodes can be more general than this 
    # approach.

    for node in nodes:

        deleted_nodes = [node]
        S = get_S(edges, deleted_nodes)

        scenarios.append(S)

    return scenarios

In [126]:
scenarios = generate_scenarios(pairs, altruistic_donors, edges)

In [127]:
k = 3

In [ ]:
# Create the loop!

# Create Xpress Model
# Initialize the model
prob = xp.problem()

# Define decision variables for each edge
y = {e: xp.var(vartype=xp.binary, name=f"y_{e[0]}_{e[1]}") for e in edges}
prob.addVariable(list(y.values()))

# Define decision variables for second stage (after node deletion)
x = {(e,s) : xp.var(vartype=xp.binary, name=f"x_{e[0]}_{e[1]}_{s}") 
     for e in edges for s,_ in enumerate(scenarios)}


prob.addVariable(list(x.values()))

# Objective: Maximize total benefit
# prob.setObjective(xp.Sum(y[e] * w for e, w in edges.items()), sense=xp.maximize)

objective = xp.Sum(w*x[e,s]/len(scenarios) for e, w in edges.items() for s, _ in enumerate(scenarios))

# print(objective)

prob.setObjective(objective, sense=xp.maximize)

# Constraints
for v in pairs:
    prob.addConstraint(xp.Sum(y[e] for e in edges if e[0] == v) <= xp.Sum(y[e] for e in edges if e[1] == v))
    prob.addConstraint(xp.Sum(y[e] for e in edges if e[1] == v) <= 1)

    for s, _ in enumerate(scenarios):
        prob.addConstraint(xp.Sum(x[(e,s)] for e in edges if e[0] == v) <= xp.Sum(x[e,s] for e in edges if e[1] == v))
        prob.addConstraint(xp.Sum(x[(e,s)] for e in edges if e[1] == v) <= 1)

for a in altruistic_donors:
    for s, _ in enumerate(scenarios):
        prob.addConstraint(xp.Sum(x[e,s] for e in edges if e[0] == a) <= 1)

for s, S in enumerate(scenarios):
    for e in edges:
        prob.addConstraint(x[e,s] <= S[e]*y[e])

finished = False # A flag to mark the end of the optimization.
infeasible = False # A (currently unused) flag to mark no feasible solution.
# TODO: Add a catch for no feasible solution. This shouldn't happen but it is
# probably better to be robust about this.

prob.controls.outputlog = 0 # This just makes it quiet to run

Gstart_time = time.time()


# prob.solve()

while finished == False and infeasible == False:
    

    start_time = time.time()


    # Solve the model
    
    prob.solve()
    opt_sol = prob.getSolution()
    
    # print(f"Solver took {time.time()-start_time} seconds to run")
    start_time = time.time()

    # Construct the graph from the optimal solution:
    DG = nx.DiGraph()
    selected_edges = [list(edges.keys())[i] for i, e in enumerate(list(edges.keys())) if opt_sol[i]>0.5]
    DG.add_edges_from(selected_edges)

    # Check if there is a cycle length that is too long:
    cycles = list(nx.simple_cycles(DG))
    
    # print(f"Finding the cycles took {time.time()-start_time} seconds")
    start_time = time.time()

    # If ok, report done.
    # TODO: Rewrite this so that it is with the max_cycle bit...
    if cycles==[] or max(map(len,cycles))<=k:
        print("")
        print("##########################################################")
        print(f"OPTIMIZATION COMPLETE: no cycles of length more than {k}.")
        print("##########################################################")
        print("")
        finished = True
        break
    
    # If not done, report that reoptimization is required:
    else:
        print("")
        print("#################################################################")
        print(f"REOPTIMIZATION REQUIRED: proposed solution contains long cycles.")
        print("#################################################################")
        print("")

    # Take the long cycle we found and make a note of its edges:
    max_cycle = max(cycles,key=len)
    cycle_edges = [(max_cycle[i],max_cycle[i+1]) for i in range(len(max_cycle)-1)]
    cycle_edges += [(max_cycle[-1],max_cycle[0])]

    # Add the constraint to remove this as an option: 
    prob.addConstraint(xp.Sum(y[e] for e in cycle_edges) <= len(max_cycle)-1)



print(f"The optimization took {time.time() - Gstart_time} seconds to execute.")




# Print the output
print("")
print("")
print("")

# print("Optimal Matches:")
# for (u, v), var in y.items():
#     if prob.getSolution(var) > 0.5:
#         print(f"{u} donates to {v} with benefit {edges[(u,v)]}")

solution_edges = [e for e in edges if prob.getSolution(y[e]) > 0.05]

print(f"Total Benefit: {prob.getObjVal()}")
print("")
print("")
print("")


##########################################################
OPTIMIZATION COMPLETE: no cycles of length more than 3.
##########################################################

The optimization took 0.008709430694580078 seconds to execute.



Total Benefit: 9.4





In [129]:
solution_edges

[('NDD1', 'P1'), ('P1', 'P2'), ('P2', 'P3'), ('P3', 'P4')]

In [130]:
prob.getSolution(x)

{(('NDD1', 'P1'), 0): -0.0,
 (('NDD1', 'P1'), 1): 1.0,
 (('NDD1', 'P1'), 2): 1.0,
 (('NDD1', 'P1'), 3): 1.0,
 (('NDD1', 'P1'), 4): 1.0,
 (('NDD1', 'P1'), 5): -0.0,
 (('P1', 'P2'), 0): -0.0,
 (('P1', 'P2'), 1): -0.0,
 (('P1', 'P2'), 2): 1.0,
 (('P1', 'P2'), 3): 1.0,
 (('P1', 'P2'), 4): 1.0,
 (('P1', 'P2'), 5): -0.0,
 (('P2', 'P3'), 0): -0.0,
 (('P2', 'P3'), 1): -0.0,
 (('P2', 'P3'), 2): -0.0,
 (('P2', 'P3'), 3): 1.0,
 (('P2', 'P3'), 4): 1.0,
 (('P2', 'P3'), 5): -0.0,
 (('P3', 'P4'), 0): -0.0,
 (('P3', 'P4'), 1): -0.0,
 (('P3', 'P4'), 2): -0.0,
 (('P3', 'P4'), 3): -0.0,
 (('P3', 'P4'), 4): 1.0,
 (('P3', 'P4'), 5): -0.0,
 (('P4', 'P5'), 0): -0.0,
 (('P4', 'P5'), 1): -0.0,
 (('P4', 'P5'), 2): -0.0,
 (('P4', 'P5'), 3): -0.0,
 (('P4', 'P5'), 4): -0.0,
 (('P4', 'P5'), 5): -0.0}